In [17]:
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import soundfile as sf
import os

SAMPLING_RATE = 16000
N_MELS = 64

SEGMENT = 252
PATIENT_ID = "00001463-100507"
EVENT = "apnea"  # or "non_apnea"

DATASET_DIR = "D:/apneaCleaned/PSG-AUDIO/APNEA_EDF"
SPECTROGRAM_DIR = "D:/apneaSpectrograms"
FETCHED_REC_DIR = "fetched_recordings"

SEG_NAME = f"seg_{SEGMENT:03d}"
SEG_DIR = os.path.join(FETCHED_REC_DIR, PATIENT_ID, EVENT, SEG_NAME)
os.makedirs(SEG_DIR, exist_ok=True)
print(f"Segment directory: {SEG_DIR}")

# fft analysis + save plot
def analyze_fft(y, label, sr=SAMPLING_RATE, seg_dir=SEG_DIR):
    fft_spectrum = np.abs(np.fft.rfft(y))
    freqs = np.fft.rfftfreq(len(y), d=1/sr)

    plt.figure(figsize=(10, 4))
    plt.semilogy(freqs, fft_spectrum)
    plt.title(f"FFT Spectrum — {label}")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Amplitude")
    plt.grid(True)
    fft_path = os.path.join(seg_dir, f"{label}_fft.png")
    plt.savefig(fft_path, dpi=200)
    plt.close()
    print(f"Saved FFT plot: {fft_path}")

    print(f"\nTop 10 frequencies in {label}:")
    top = np.argsort(fft_spectrum)[-10:][::-1]
    for i in top:
        print(f"{freqs[i]:6.1f} Hz  |  Amp: {fft_spectrum[i]:.2f}")

# plot waveform and save to fetched_recordings
def plot_waveform(y, label, sr=SAMPLING_RATE, segment_dir=SEG_DIR):
    plt.figure(figsize=(10, 3))
    librosa.display.waveshow(y, sr=sr)
    plt.title(f"Waveform — {label}")
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")

    if label == "original_segment":
        waveplot_path = os.path.join(segment_dir, "original_waveform.png")
    elif label == "recorded_segment_raw":
        waveplot_path = os.path.join(segment_dir, "recorded_waveform_raw.png")

    plt.savefig(waveplot_path, dpi=200)
    plt.close()
    print(f"Saved waveform plot: {waveplot_path}")

# plot spec
def plot_spec(y, label, seg_dir=SEG_DIR, sr=SAMPLING_RATE, n_mels=N_MELS):
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    S_db = librosa.power_to_db(S, ref=np.max) # we need amplitude scale

    # save to fetched_recordings
    if label == "recorded_segment_raw":
        np.save(os.path.join(seg_dir, "recorded_spectrogram_raw.npy"), S_db)
        spec_path = os.path.join(seg_dir, "recorded_spectrogram_raw.png")
    else:
        np.save(os.path.join(seg_dir, f"{label}.npy"), S_db)
        spec_path = os.path.join(seg_dir, f"{label}.png")

    plt.figure(figsize=(6, 4))
    librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel Spectrogram (dB) — {label}")
    plt.savefig(spec_path, dpi=200)
    plt.close()
    print(f"Saved spectrogram: {spec_path}")

def process_segment(y, label, seg_dir=SEG_DIR, sr=SAMPLING_RATE):
    wav_path = os.path.join(seg_dir, f"{label}.wav")
    # save to fetched_recordings
    sf.write(wav_path, y.astype(np.float32), sr)
    print(f"Saved audio: {wav_path}")

    analyze_fft(y, label)
    plot_waveform(y, label)

    if label != "original_segment":
        plot_spec(y, label)

    # display audio
    ipd.display(ipd.Audio(y, rate=sr))

# PROCESS ORIGINAL SEGMENT
# if apnea, process '_ap.npy', if non_apnea, process '_nap.npy'
if EVENT == "apnea":
    original_npy = os.path.join(DATASET_DIR, PATIENT_ID, f"{PATIENT_ID}_ap.npy")
else:
    original_npy = os.path.join(DATASET_DIR, PATIENT_ID, f"{PATIENT_ID}_nap.npy")

all_segments = np.load(original_npy) # load all apneic segments of a patient
y_orig = all_segments[SEGMENT] # extract the necessary one

process_segment(y_orig, "original_segment")

# LOAD AND PROCESS ORIGINAL SEGMENT
pre_spec_path = os.path.join(SPECTROGRAM_DIR, PATIENT_ID, EVENT, f"{SEG_NAME}.npy")
pre_spec = np.load(pre_spec_path)
np.save(os.path.join(SEG_DIR, "original_spectrogram.npy"), pre_spec)

plt.figure(figsize=(6, 4))
librosa.display.specshow(pre_spec, sr=SAMPLING_RATE, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title("Mel Spectrogram — Original")
plt.savefig(os.path.join(SEG_DIR, "original_spectrogram.png"), dpi=200)
plt.close()
print("Saved original spectrogram image in png.")

# PROCESS RAW RECORDED SEGMENT
recorded_wav = os.path.join(SEG_DIR, "recorded_segment_raw.wav")
y_rec, _ = librosa.load(recorded_wav, sr=SAMPLING_RATE)
process_segment(y_rec, "recorded_segment_raw")

Segment directory: fetched_recordings\00001463-100507\apnea\seg_252
Saved audio: fetched_recordings\00001463-100507\apnea\seg_252\original_segment.wav
Saved FFT plot: fetched_recordings\00001463-100507\apnea\seg_252\original_segment_fft.png

Top 10 frequencies in original_segment:
   9.5 Hz  |  Amp: 149.68
  48.6 Hz  |  Amp: 132.15
  23.4 Hz  |  Amp: 99.55
  16.9 Hz  |  Amp: 73.95
   9.4 Hz  |  Amp: 65.50
  16.4 Hz  |  Amp: 58.43
  16.2 Hz  |  Amp: 56.73
  18.4 Hz  |  Amp: 52.65
  15.0 Hz  |  Amp: 51.64
  23.3 Hz  |  Amp: 51.52
Saved waveform plot: fetched_recordings\00001463-100507\apnea\seg_252\original_waveform.png


Saved original spectrogram image in png.
Saved audio: fetched_recordings\00001463-100507\apnea\seg_252\recorded_segment_raw.wav
Saved FFT plot: fetched_recordings\00001463-100507\apnea\seg_252\recorded_segment_raw_fft.png

Top 10 frequencies in recorded_segment_raw:
  50.1 Hz  |  Amp: 130.16
  50.0 Hz  |  Amp: 73.07
   0.0 Hz  |  Amp: 42.17
  50.2 Hz  |  Amp: 35.56
  49.9 Hz  |  Amp: 31.50
 150.1 Hz  |  Amp: 29.05
  18.9 Hz  |  Amp: 28.07
  16.7 Hz  |  Amp: 27.97
  20.4 Hz  |  Amp: 26.99
  13.8 Hz  |  Amp: 25.68
Saved waveform plot: fetched_recordings\00001463-100507\apnea\seg_252\recorded_waveform_raw.png
Saved spectrogram: fetched_recordings\00001463-100507\apnea\seg_252\recorded_spectrogram_raw.png
